In [9]:
cd("/home/jm2386/Active_Lattice/")
using DrWatson
@quickactivate "Active_Lattice"
include("/home/jm2386/Active_Lattice/src/2d_pdes.jl");
include("/home/jm2386/Active_Lattice/src/2d_sims.jl");
include("/home/jm2386/Active_Lattice/src/2d_plot.jl");

v1.0
v1.1


v2.1


In [12]:
#find wave parameters
DT, DR, N, Nθ, Δx, Lx, Ly, δt, δ, ϕa, v0 = (1.0, 1.0, 32, 20, 0.2, 10.0, 10.0, 1e-5, 0.1, 0.5, 10.0);
T, save_interval, param_name = (20.0, 0.1, "test")
Params = []
map(0.34:0.02:0.4) do ϕp
    param = _2d_new_param(DT, v0, DR, N, Nθ, Δx, Lx, Ly, ϕa, ϕp, δt, δ; T = T, name = param_name, save_interval = save_interval, save_on = true)
    push!(Params,param)
end
values = map(lin_pert_values,Params)

4-element Vector{Tuple{ComplexF64, Vector{ComplexF64}}}:
 (0.14222650742100668 + 8.149275261526953e-17im, [0.7122030741380849 + 0.0im, -0.536344723422986 - 2.152556068408315e-16im, 2.3422189026442213e-16 - 0.4519052257689367im, -0.029698385720590068 - 3.779609537637431e-17im, 1.0444840160404859e-17 + 0.0008896202508392547im, 1.5122215893318102e-5 + 1.8616181363726077e-19im, 2.787083432457926e-21 - 1.651820153599327e-7im, -1.255735019577249e-9 - 4.2528459519802045e-23im, -2.385815416304108e-25 + 7.022813786180575e-12im, 3.009612523523634e-14 + 1.9750321042084604e-27im, 6.859450555831123e-30 - 1.0196765639599794e-16im])
 (0.06963230988203653 + 0.03096160537067711im, [0.8949783535520066 + 0.0im, -0.35891847396277876 - 0.09086868043737224im, 0.11118888828488127 - 0.22218627026293478im, -0.012830428093278747 - 0.006300285020070033im, -0.00016234421573825558 + 0.0003334728171834216im, 4.896339627920894e-6 + 2.372045142594297e-6im, 2.2263849261237436e-8 - 4.6101537448392646e-8im, -3.017560607

In [4]:
param = Params[2]
ts, fs = load_compress_pde(param);

MethodError: MethodError: no method matching getindex(::typeof(params), ::Int64)

In [2]:
# create fig
    using PyPlot
    PyPlot.close("all")
    rc("text", usetex=true)
    fig = plt.figure(figsize=(10,5))
# add image
    ax      = fig.add_axes([0.1, 0.1, 0.3, 0.6])
    cbar_ax = fig.add_axes([0.41, 0.1, 0.02, 0.6])
    plot_pde_rho(ax, cbar_ax, param, f; cmin = 0.0, cmax = 1.0, cbar = true)
    ax      = fig.add_axes([0.55, 0.1, 0.3, 0.6])
    cbar_ax = fig.add_axes([0.86, 0.1, 0.02, 0.6])
    plot_pde_mag(ax, cbar_ax, param, f; cmin = -0.4, cmax = 0.4, cbar = true, density = 0.5)
#
display(fig)

UndefVarError: UndefVarError: param not defined

In [61]:
using PyCall
@pyimport matplotlib.animation as anim

fig, axs = plt.subplots(1, 1, figsize=(5,5))
@unpack save_interval = param
scale_factor = 10
interval  = save_interval*scale_factor*1000
frames    = Int64( length(ts)/scale_factor ÷ 1)
function makeframe(i)
        clf()
        ax = fig.add_subplot(111)
        t, f = ts[i+1], fs[i+1]
        # add image
                ax      = fig.add_axes([0.1, 0.1, 0.3, 0.6])
                cbar_ax = fig.add_axes([0.41, 0.1, 0.02, 0.6])
                plot_pde_rho(ax, cbar_ax, param, f; cmin = 0.0, cmax = 1.0, cbar = true)
                ax      = fig.add_axes([0.55, 0.1, 0.3, 0.6])
                cbar_ax = fig.add_axes([0.86, 0.1, 0.02, 0.6])
                plot_pde_mag(ax, cbar_ax, param, f; cmin = -0.4, cmax = 0.4, cbar = true, density = 0.5)
        #
        return fig
end
myanim = anim.FuncAnimation(fig, makeframe, frames=frames, interval=interval)
# Convert it to an MP4 movie file and saved on disk in this format.
        filename, pathname = pde_vid_save_name(param,ts[end])
        mkpath(pathname)
        myanim[:save](filename, bitrate=-1, dpi= 100, extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])